In [1]:
import utils 
from utils import *

Using TensorFlow backend.


In [93]:
# plt.rcParams['figure.figsize'] = 10, 10
# %matplotlib inline
exp_name = "_5fold_fcn"

### Load Data

In [94]:
train = pd.read_json("data/input/train.json")
target_train=train['is_iceberg']
test = pd.read_json("data/input/test.json")


In [95]:
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
X_test_angle=test['inc_angle']

In [ ]:
# sample_weights = {0:}

### Preprocess

In [96]:


#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

### Train Model 

In [97]:


batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

In [98]:

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=5):
    es = EarlyStopping('val_loss', patience=5, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
#     lrplateu = ReduceLROnPlateau(monitor='val_loss',  patience=2, verbose=1, factor=0.5, min_lr=1e-6)

    return [es, msave]


In [16]:
def getModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def trainKfold(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=17).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "weights/{}_{}.hdf5".format(exp_name,j+1)
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        
        model= getModel()
        model.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        model.load_weights(filepath=file_path)
        #Getting Training Score
        score = model.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        
        #Getting Test Score
        score = model.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=model.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=model.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=model.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    
    train_log_loss = log_loss(target_train, y_train_pred_log) 
    valid_log_loss = log_loss(target_train, y_valid_pred_log)
    
    print('\n Train Log Loss Validation= ',train_log_loss)
    print(' Valid Log Loss Validation= ',valid_log_loss)
    
    
    return y_train_pred_log, y_valid_pred_log, y_test_pred_log, train_log_loss, valid_log_loss





In [3]:
# train_preds , val_preds, test_preds, train_log_loss,valid_log_loss = trainKfold(X_train, X_angle, X_test)

In [62]:
train_preds , val_preds, test_preds, train_log_loss, valid_log_loss = pickle.load(file=open("tmp_results.dmp","rb"))


    

In [63]:
train_preds  = pd.DataFrame(data={"is_iceberg":train_preds})
train_preds["id"]=train['id'].astype(str)
train_preds.set_index("id",inplace=True)




In [45]:
val_preds  = pd.DataFrame(data={"id":train['id'],"is_iceberg":val_preds})
val_preds["id"]=train['id'].astype(str)
val_preds.set_index("id",inplace=True)

### Submission

In [67]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id'].astype(str)
submission['is_iceberg']=test_preds
submission.to_csv('subm/{}.csv'.format(exp_name), index=False)

submission.set_index("id",inplace=True)

In [12]:

def submit_and_update1(phase, exp_name, train_preds, val_preds,test_preds, train_log_loss,valid_log_loss):

#     lb  = 0.0
#     try:
#         submission_output = subprocess.check_output(["kg", "submit",'subm/{}.csv'.format(exp_name)])
#         lb = float(submission_output)

#     except:
#         print (submission_output)


    update_results_h5(phase,exp_name,train_preds,val_preds,test_preds,train_log_loss,valid_log_loss,LB_score=lb)

#     subprocess.call(["sudo", "shutdown", "now"])


In [74]:
def update_results_h5(phase, exp_name, train_preds, val_preds,test_preds, train_log_loss,valid_log_loss, LB_score=0.0 ):
        store = pd.HDFStore('data/results.h5')
        store.append("/{}/train/{}".format(phase,exp_name),train_preds )
        store.append("/{}/valid/{}".format(phase,exp_name),val_preds)
        store.append("/{}/test/{}".format(phase,exp_name),test_preds)  
                     
        store.append("/summary",pd.DataFrame(data={"phase":[phase],
                                                   "exp":[exp_name],
                                                   "train_log_loss":[train_log_loss],
                                                   "val_log_loss":[valid_log_loss], 
                                                   "LB":[LB_score] }) )  

        store.close()

In [76]:
update_results_h5("ph1",exp_name,train_preds,val_preds,submission,train_log_loss,valid_log_loss,0.318)

In [77]:
store = pd.HDFStore('data/results.h5')


In [45]:
np.corrcoef(store.select("/ph1/valid/_5fold_baseline").values.ravel(), store.select("/ph1/train/_5fold_baseline").values.ravel())

array([[ 1.        ,  0.80484443],
       [ 0.80484443,  1.        ]])

In [79]:
store.keys()

['/summary',
 '/ph1/test/_5fold_baseline',
 '/ph1/train/_5fold_baseline',
 '/ph1/valid/_5fold_baseline']

In [82]:
benchmark3cv = pd.read_csv("subm/sub_benchmark.csv")

In [84]:
benchmark3cv.set_index('id',inplace=True)

In [92]:
benchmark3cv.join(store.get("/ph1/test/_5fold_baseline"),lsuffix="_3cv").corr()

,is_iceberg_3cv,is_iceberg
is_iceberg_3cv,1.000000,0.975071
is_iceberg,0.975071,1.000000
